# 第六章 决策树

## 决策树的训练和可视化

In [1]:
from sklearn.datasets import load_iris

In [2]:
from sklearn.tree import DecisionTreeClassifier

In [3]:
iris = load_iris()

In [4]:
X = iris.data[:, 2:]

In [5]:
X[:5]

array([[1.4, 0.2],
       [1.4, 0.2],
       [1.3, 0.2],
       [1.5, 0.2],
       [1.4, 0.2]])

In [6]:
y = iris.target

In [7]:
tree_clf = DecisionTreeClassifier(max_depth=2)

In [8]:
tree_clf.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

- 要将决策树可视化，首先使用export_graphviz()方法输出一个图形定义文件，.dot

In [9]:
from sklearn.tree import export_graphviz

In [10]:
export_graphviz(
    tree_clf,
    out_file=("./iris_tree.dot"),
    feature_names=iris.feature_names[2:],
    class_names=iris.target_names,
    rounded=True,
    filled=True
)

然后转换dot到png：

`dot -Tpng iris_tree.dot -o iris_tree.png`

gini不纯度的计算公式：

1 - 求和【(类别为k的训练实例的占比)^2】

In [11]:
1 - (0/54)**2 - (49/54)**2 - (5/54)**2

0.1680384087791495

信息熵计算公式：

-求和【(类别为k的训练实例占比)*log(类别为k的训练实例占比)】

In [12]:
from math import log

In [13]:
-(49/54*log(49/54) + 5/54*log(5/54))

0.30849545083110386

## 估算类别概率

In [14]:
tree_clf.predict_proba([[5, 1.5]])

array([[0.        , 0.90740741, 0.09259259]])

In [15]:
tree_clf.predict([[5, 1.5]])

array([1])

In [16]:
from sklearn.tree import DecisionTreeRegressor

In [17]:
tree_reg = DecisionTreeRegressor(max_depth=2)

In [18]:
tree_reg.fit(X, y)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

## 不稳定性

- 决策树青睐正交的决策边界（所有分裂都与轴线垂直），这导致它们对训练集的旋转非常敏感。

- 更概括的说，决策树的主要问题是它们对训练数据中的小变化非常敏感。

- 限制这种问题的方法之一是使用PCA。

练习8

In [19]:
from sklearn.datasets import make_moons

In [20]:
X, y = make_moons(n_samples=10000, noise=0.4)

In [21]:
X[:5]

array([[ 0.6973432 ,  0.68765165],
       [-1.42934965,  1.48313622],
       [-0.71295152,  0.71604384],
       [ 0.43646406, -0.44608182],
       [-0.19845559,  0.71902177]])

In [22]:
y[:5]

array([0, 0, 0, 1, 1], dtype=int64)

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [25]:
tree_clf = DecisionTreeClassifier()

In [27]:
tree_clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [28]:
tree_clf.score(X_test, y_test)

0.7956666666666666

In [29]:
param = {
    "max_depth": [2, 3, 4, 5, 6, 7, 8, 9, 10],
    "max_leaf_nodes": [10, 12, 16, 18, 20]
}

In [30]:
from sklearn.model_selection import GridSearchCV

In [31]:
gs = GridSearchCV(tree_clf, param_grid=param, cv=5, return_train_score=True, n_jobs=-1)

In [32]:
gs.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=None,
                                              splitter='best'),
             iid='warn', n_jobs=-1,
             param_grid={'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'max_leaf_nodes': [10, 12, 16, 18, 20]},
    

In [33]:
gs.best_estimator_

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
                       max_features=None, max_leaf_nodes=10,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [34]:
gs.best_params_

{'max_depth': 2, 'max_leaf_nodes': 10}

In [37]:
best_model = gs.best_estimator_

In [38]:
best_model.score(X_test, y_test)

0.85

习题8

In [39]:
from sklearn.model_selection import ShuffleSplit

In [40]:
n_trees = 1000
n_instances = 100

In [41]:
mini_sets = []

In [42]:
len(X_train)

7000

In [43]:
rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances, random_state=42)

In [47]:
for mini_train_index, mini_test_index in rs.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

In [54]:
from sklearn.base import clone
from sklearn.metrics import accuracy_score

In [51]:
forest = [clone(best_model) for _ in range(n_trees)]

In [52]:
accuracy_scores = []

In [55]:
for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)
    
    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))

In [57]:
import numpy as np

In [58]:
np.mean(accuracy_scores)

0.8193336666666666

In [59]:
Y_pred = np.empty([n_trees, len(X_test)], dtype=np.uint8)

In [60]:
Y_pred

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [61]:
for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)

In [64]:
Y_pred.shape

(1000, 3000)

In [65]:
Y_pred

array([[0, 0, 1, ..., 0, 1, 1],
       [0, 0, 1, ..., 1, 1, 1],
       [0, 0, 1, ..., 1, 1, 1],
       ...,
       [0, 0, 1, ..., 1, 1, 1],
       [0, 0, 1, ..., 0, 1, 1],
       [0, 0, 1, ..., 0, 1, 1]], dtype=uint8)

In [66]:
from scipy.stats import mode

In [67]:
y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

In [77]:
mode(Y_pred, axis=0)

ModeResult(mode=array([[0, 0, 1, ..., 1, 1, 1]], dtype=uint8), count=array([[1000,  997,  997, ...,  714,  991,  691]]))

In [69]:
y_pred_majority_votes.shape

(1, 3000)

In [71]:
n_votes.shape

(1, 3000)

In [72]:
y_pred_majority_votes

array([[0, 0, 1, ..., 1, 1, 1]], dtype=uint8)

In [73]:
n_votes

array([[1000,  997,  997, ...,  714,  991,  691]])

In [74]:
accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

0.8473333333333334